In [1]:
--Heap typeclass
module Heap where

class Heap h where
    empty :: h a
    isEmpty :: h a -> Bool
    insert :: Ord a => a -> h a -> h a
    merge :: Ord a => h a -> h a -> h a
    findMin :: Ord a => h a -> a
    deleteMin :: Ord a => h a -> h a

In [2]:
:l Heap.hs

In [3]:
--LeftiestHeap
module LeftiestHeap where

import Heap

data LHeap a = 
    Empty |
    Tree {
        _rank :: !Int,
        _value :: !a,
        _left :: !(LHeap a),
        _right :: !(LHeap a)
    } deriving (Show)
    
rank :: LHeap a -> Int
rank Empty = 0
rank (Tree r _ _ _) = r

singleton :: a -> LHeap a
singleton x = Tree 1 x Empty Empty

instance Heap LHeap where
    empty = Empty
    
    isEmpty Empty = True
    isEmpty _ = False
   
    insert  x = merge (singleton x)
   
    merge Empty h = h
    merge h Empty = h
    merge h1@(Tree r1 v1 left1 right1) h2@(Tree r2 v2 left2 right2)
        | v1 <= v2 = makeT v1 left1 (merge right1 h2)
        |otherwise = makeT v2 left2 (merge right2 h1)
        where
            makeT :: a -> LHeap a -> LHeap a -> LHeap a
            makeT v lt rt = let 
                r1 = rank lt
                r2 = rank rt
                in
                if r1 <= r2 then Tree (r1+1) v rt lt else Tree (r2+1) v lt rt
    
    findMin Empty = error "Error: Empty Heap"
    findMin (Tree _ v _ _) = v
    
    deleteMin Empty = error "Error: Empty Heap"
    deleteMin (Tree r v lt rt) = merge lt rt

In [4]:
--Exercise 3.2
:l LeftiestHeap.hs
import LeftiestHeap

directInsert :: Ord a => a -> LHeap a -> LHeap a
directInsert x Empty = Tree 1 x Empty Empty
directInsert x h@(Tree r v lt rt) 
    | x < v = Tree 1 x h Empty
    | otherwise = Tree r v (directInsert x lt) rt
    
main :: IO ()
main = do
    let insertResult = insert 1 Empty
    let directInsertResult = directInsert 1 Empty 
    print insertResult
    print directInsertResult
    
main

Tree {_rank = 1, _value = 1, _left = Empty, _right = Empty}
Tree {_rank = 1, _value = 1, _left = Empty, _right = Empty}

In [5]:
--Exercise 3.3
:l LeftiestHeap.hs
import LeftiestHeap
import Data.List

fromList :: Ord a => [a] -> LHeap a
fromList lst = fromList' (map singleton lst)
    where 
        fromList' [t] = t
        fromList' (x : y : xs) = fromList' $ pair (x : y: xs)
        pair (x : y : xs) = merge x y : pair xs
        pair xs = xs
        
main :: IO ()
main = do
    let heapFromList = fromList [2,1,4,5,3]
    print heapFromList
    
main

Tree {_rank = 2, _value = 1, _left = Tree {_rank = 1, _value = 2, _left = Tree {_rank = 1, _value = 3, _left = Empty, _right = Empty}, _right = Empty}, _right = Tree {_rank = 1, _value = 4, _left = Tree {_rank = 1, _value = 5, _left = Empty, _right = Empty}, _right = Empty}}

In [6]:
--Exercise 3.4
--Weight-biased leftiest heap
module WeightedHeap where

import Heap

data WHeap a =
    Empty |
    Tree {
        _size :: !Int,
        _value :: !a,
        _left :: WHeap a,
        _right :: WHeap a
    } deriving (Show)
    
size :: WHeap a -> Int
size Empty = 0
size (Tree s _ _ _) = s

singleton :: a -> WHeap a
singleton x = Tree 1 x Empty Empty

instance Heap WHeap where
    empty = Empty
    
    isEmpty Empty = True
    isEmpty _ = False
   
    insert  x = merge (singleton x)
   
    merge Empty h = h
    merge h Empty = h
    merge h1@(Tree s1 v1 left1 right1) h2@(Tree s2 v2 left2 right2)
        | v1 <= v2 =
            if size left1 >= size right1 + s2
            then Tree (s1+s2) v1 left1 (merge right1 h2)
            else Tree (s1+s2) v1 (merge right1 h2) left1
        |otherwise = merge h2 h1
    
    findMin Empty = error "Error: Empty Heap"
    findMin (Tree _ v _ _) = v
    
    deleteMin Empty = error "Error: Empty Heap"
    deleteMin (Tree r v lt rt) = merge lt rt

In [7]:
--Binomial heap
module BinomialHeap where

import Heap

data BTree a = 
    Node {
        _rank :: !Int,
        _value :: !a,
        _children :: [BTree a]
    } deriving (Show)
    
newtype BHeap a = BH [BTree a] deriving (Show) 
    
link :: Ord a => BTree a -> BTree a -> BTree a
link t1@(Node r v1 c1) t2@(Node _ v2 c2) 
    | v1 < v2 = Node (r+1) v1 (t2 : c1)
    | otherwise = Node (r+1) v2 (t1 : c2)
    
singleton :: a -> BTree a
singleton x = Node 0 x []

rank :: BTree a -> Int
rank (Node r _ _) = r

root :: BTree a -> a
root (Node _ x _) = x

children :: BTree a -> [BTree a]
children (Node _ _ c) = c

insTree :: Ord a => BTree a -> [BTree a] -> [BTree a]
insTree t [] = [t]
insTree t ts@(t' : ts')
    | rank t < rank t' = t : ts
    | otherwise = insTree (link t t') ts' --Always assume rank(t) <= rank(t') !!

removeMinTree :: Ord a => [BTree a] -> (BTree a, [BTree a])
removeMinTree [t] = (t, [])
removeMinTree (t : ts) = let
    (t', ts') = removeMinTree ts
    in
    if root t <= root t' 
    then (t, ts)
    else (t', t : ts')

insertT :: Ord a =>a -> [BTree a] -> [BTree a]
insertT x = insTree (singleton x)

mergeT :: Ord a => [BTree a] -> [BTree a] -> [BTree a]
mergeT [] ts = ts
mergeT ts [] = ts
mergeT ts1@(t1 : ts1') ts2@(t2 : ts2') 
    | rank t1 < rank t2 = t1 : mergeT ts1' ts2
    | rank t2 > rank t1 = t2 : mergeT ts1 ts2'
    | otherwise = insTree (link t1 t2) (mergeT ts1' ts2')

deleteMinT :: Ord a => [BTree a] -> [BTree a]
deleteMinT = snd . removeMinTree

fmap' :: ([BTree a] -> [BTree b]) -> BHeap a -> BHeap b
fmap' f (BH ts) = BH (f ts)

fmap2' :: ([BTree a] -> [BTree b] -> [BTree c]) -> BHeap a -> BHeap b -> BHeap c
(fmap2' f) (BH ts1) (BH ts2) = BH (f ts1 ts2)

instance Heap BHeap where
    empty = BH []
    
    isEmpty (BH []) = True
    isEmpty _ = False

    --insert :: Ord a => a -> BHeap a -> BHeap a
    insert x = fmap' (insertT x)

    --merge :: Ord a => BHeap a -> BHeap a -> BHeap a
    merge = fmap2' mergeT 

    --findMin :: Ord a => BHeap a -> a
    findMin (BH ts) = root . fst . removeMinTree $ ts

    --deleteMin :: Ord a => BHeap a -> BHeap a
    deleteMin = fmap' deleteMinT

In [8]:
--Exercise 3.5
:l BinomialHeap.hs
import BinomialHeap

directFindMin :: Ord a => BHeap a -> a
directFindMin (BH [t]) = root t
directFindMin h@(BH (t : ts)) = let
    v = root t 
    v' = directFindMin (BH ts)
    in
    if v < v' then v else v'

In [5]:
--Exercise 3.6
module RankBinomialHeap where

import Heap

data BTree a = 
    Node {
        _value :: !a,
        _children :: [BTree a]
    } deriving (Show)
    
type RBTree a = (Int, BTree a)
newtype RBHeap a = RBH [RBTree a] deriving (Show) 
    
linkT :: Ord a => BTree a -> BTree a -> BTree a
linkT t1@(Node v1 c1) t2@(Node v2 c2) 
    | v1 < v2 = Node v1 (t2 : c1)
    | otherwise = Node v2 (t1 : c2)
    
link :: Ord a => RBTree a -> RBTree a -> RBTree a
link (r, t1) (_, t2) = (r+1, linkT t1 t2)
    
singletonT :: a -> BTree a
singletonT x = Node x []

singleton :: a -> RBTree a
singleton x = (0, singletonT x)

rank :: RBTree a -> Int
rank = fst

rootT :: BTree a -> a
rootT (Node x _) = x

root :: RBTree a -> a
root = rootT . snd

childrenT :: BTree a -> [BTree a]
childrenT (Node _ c) = c

children :: RBTree a -> [BTree a]
children = childrenT . snd

insTree :: Ord a => RBTree a -> [RBTree a] -> [RBTree a]
insTree t [] = [t]
insTree t ts@(t' : ts')
    | rank t < rank t' = t : ts
    | otherwise = insTree (link t t') ts' --Always assume rank(t) <= rank(t') !!

removeMinTree :: Ord a => [RBTree a] -> (RBTree a, [RBTree a])
removeMinTree [t] = (t, [])
removeMinTree (t : ts) = let
    (t', ts') = removeMinTree ts
    in
    if root t <= root t' 
    then (t, ts)
    else (t', t : ts')

insertT :: Ord a =>a -> [RBTree a] -> [RBTree a]
insertT x = insTree (singleton x)

mergeT :: Ord a => [RBTree a] -> [RBTree a] -> [RBTree a]
mergeT [] ts = ts
mergeT ts [] = ts
mergeT ts1@(t1 : ts1') ts2@(t2 : ts2') 
    | rank t1 < rank t2 = t1 : mergeT ts1' ts2
    | rank t2 > rank t1 = t2 : mergeT ts1 ts2'
    | otherwise = insTree (link t1 t2) (mergeT ts1' ts2')

deleteMinT :: Ord a => [RBTree a] -> [RBTree a]
deleteMinT = snd . removeMinTree

fmap' :: ([RBTree a] -> [RBTree b]) -> RBHeap a -> RBHeap b
fmap' f (RBH ts) = RBH (f ts)

fmap2' :: ([RBTree a] -> [RBTree b] -> [RBTree c]) -> RBHeap a -> RBHeap b -> RBHeap c
(fmap2' f) (RBH ts1) (RBH ts2) = RBH (f ts1 ts2)

instance Heap RBHeap where
    empty = RBH []
    
    isEmpty (RBH []) = True
    isEmpty _ = False

    --insert :: Ord a => a -> BHeap a -> BHeap a
    insert x = fmap' (insertT x)

    --merge :: Ord a => BHeap a -> BHeap a -> BHeap a
    merge = fmap2' mergeT 

    --findMin :: Ord a => BHeap a -> a
    findMin (RBH ts) = root . fst . removeMinTree $ ts

    --deleteMin :: Ord a => BHeap a -> BHeap a
    deleteMin = fmap' deleteMinT

In [5]:
--Exercise 3.7
:l Heap.hs
:l RankBinomialHeap.hs
--module ExplicitMinBinomialHeap where

import qualified Heap as H
import qualified RankBinomialHeap as H

data EMBHeap a = 
    E |
    NE {_min :: a, _heap :: H.RBHeap a} deriving (Show)

instance Heap EMBHeap where
    empty = E
    
    isEmpty E = True
    isEmpty _ = False

    insert x E = NE x (H.insert x H.empty)
    insert x (NE cur_x h)
        | x < cur_x = NE x (H.insert x h)
        | otherwise = NE cur_x (H.insert x h) 
        
    merge E emh = emh
    merge emh E = emh
    merge (NE x1 h1) (NE x2 h2) 
        | x1 < x2 = NE x1 (H.merge h1 h2)
        | otherwise = NE x2 (H.merge h1 h2)
        
    findMin E = error "Error: Empty Heap"
    findMin (NE x _) = x
    
    deleteMin E = error "Error: Empty Heap"
    deleteMin (NE x h) = let
        h' = H.deleteMin h
        in
        if H.isEmpty h' then E else NE (findMin h) h'

In [3]:
--Red-Black Tree
module RedBlackTree where

data Color = R | B deriving (Show)
data Tree a = 
    E |
    T {
        _color :: Color,
        _left :: !(Tree a),
        _value :: !a,
        _right :: !(Tree a)
    } deriving (Show)
    
member :: Ord a => a -> Tree a -> Bool
member _ E = False
member x (T _ lt y rt) 
    | x < y = member x lt
    | x > y = member x rt
    | otherwise = True
    
insert :: Ord a => a -> Tree a -> Tree a
insert x t = let
    ins E = T R E x E
    ins t@(T c lt y rt) 
        | x < y = balance c (ins lt) y rt
        | x > y = balance c lt y (ins rt)
        | otherwise = t
    T _ lt y rt = ins t
    in
    T B lt y rt
    
balance :: Color -> Tree a -> a -> Tree a -> Tree a
balance B (T R (T R a x b) y c) z d = T R (T B a x b) y (T B c z d)
balance B (T R a x (T R b y c)) z d = T R (T B a x b) y (T B c z d)
balance B a x (T R (T R b y c) z d) = T R (T B a x b) y (T B c z d)
balance B a x (T R b y (T R c z d)) = T R (T B a x b) y (T B c z d)
balance c a x b = T c a x b

In [24]:
--Exercise 3.9
:l RedBlackTree.hs
import RedBlackTree

data Digit a = One a (Tree a) | Two a (Tree a) a (Tree a) deriving (Show)

fromOrdList :: Ord a => [a] -> Tree a
fromOrdList = foldl link E . foldr add [] 

link :: Ord a => Tree a -> Digit a -> Tree a
link treeSoFar (One a t) = T B treeSoFar a t
link treeSoFar (Two a1 t1 a2 t2) = T B (T R treeSoFar a1 t1) a2 t2 

add :: Ord a => a -> [Digit a] -> [Digit a]
add a [] = [One a E]
add a ((One a1 t1) : ds) = Two a E a1 t1 : ds
add a ((Two a1 t1 a2 t2) : ds) = One a E : (incr (One a1 (T B t1 a2 t2) : ds))
    where
        incr [] = []
        incr [d] = [d]
        incr ((One a1 t1) : (One a2 t2) : ds) = Two a1 t1 a2 t2 : ds
        incr ((One a1 t1) : (Two a2 t2 a3 t3) : ds) = (One a1 t1) : (One a2 (T B t2 a3 t3)) : incr ds
        
main :: IO ()
main = do
    mapM_ print [fromOrdList [1..n] | n <- [1..8]]
    
main

T {_color = B, _left = E, _value = 1, _right = E}
T {_color = B, _left = T {_color = R, _left = E, _value = 1, _right = E}, _value = 2, _right = E}
T {_color = B, _left = T {_color = B, _left = E, _value = 1, _right = E}, _value = 2, _right = T {_color = B, _left = E, _value = 3, _right = E}}
T {_color = B, _left = T {_color = B, _left = T {_color = R, _left = E, _value = 1, _right = E}, _value = 2, _right = E}, _value = 3, _right = T {_color = B, _left = E, _value = 4, _right = E}}
T {_color = B, _left = T {_color = R, _left = T {_color = B, _left = E, _value = 1, _right = E}, _value = 2, _right = T {_color = B, _left = E, _value = 3, _right = E}}, _value = 4, _right = T {_color = B, _left = E, _value = 5, _right = E}}
T {_color = B, _left = T {_color = R, _left = T {_color = B, _left = T {_color = R, _left = E, _value = 1, _right = E}, _value = 2, _right = E}, _value = 3, _right = T {_color = B, _left = E, _value = 4, _right = E}}, _value = 5, _right = T {_color = B, _left = E, _valu

In [25]:
--Exercise 3.10 (a)
:l RedBlackTree.hs
import RedBlackTree

insert' :: Ord a => a -> Tree a -> Tree a
insert' x t = let
    ins E = T R E x E
    ins t@(T c lt y rt) 
        | x < y = lbalance c (ins lt) y rt
        | x > y = rbalance c lt y (ins rt)
        | otherwise = t
    T _ lt y rt = ins t
    in
    T B lt y rt
    
lbalance :: Color -> Tree a -> a -> Tree a -> Tree a
lbalance B (T R (T R a x b) y c) z d = T R (T B a x b) y (T B c z d)
lbalance B (T R a x (T R b y c)) z d = T R (T B a x b) y (T B c z d)
lbalance c a x b = T c a x b

rbalance :: Color -> Tree a -> a -> Tree a -> Tree a
rbalance B a x (T R (T R b y c) z d) = T R (T B a x b) y (T B c z d)
rbalance B a x (T R b y (T R c z d)) = T R (T B a x b) y (T B c z d)
rbalance c a x b = T c a x b

In [26]:
--Exercise 3.10 (b)
:l RedBlackTree.hs
import RedBlackTree

insert'' :: Ord a => a -> Tree a -> Tree a
insert'' x t = let
    ins E = T R E x E
    ins t@(T c lt y rt) 
        | x < y = if _value (ins lt) == x then lrbalance c (ins lt) y rt else llbalance c (ins lt) y rt
        | x > y = if _value (ins rt) == y then rrbalance c lt y (ins rt) else rlbalance c lt y (ins rt)
        | otherwise = t
    T _ lt y rt = ins t
    in
    T B lt y rt
    
llbalance :: Color -> Tree a -> a -> Tree a -> Tree a
llbalance B (T R (T R a x b) y c) z d = T R (T B a x b) y (T B c z d)
llbalance c a x b = T c a x b

lrbalance :: Color -> Tree a -> a -> Tree a -> Tree a
lrbalance B (T R a x (T R b y c)) z d = T R (T B a x b) y (T B c z d)
lrbalance c a x b = T c a x b

rlbalance :: Color -> Tree a -> a -> Tree a -> Tree a
rlbalance B a x (T R (T R b y c) z d) = T R (T B a x b) y (T B c z d)
rlbalance c a x b = T c a x b

rrbalance :: Color -> Tree a -> a -> Tree a -> Tree a
rrbalance B a x (T R b y (T R c z d)) = T R (T B a x b) y (T B c z d)
rrbalance c a x b = T c a x b